In [1]:
import torch
from torch import nn

In [3]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

## Parameter Access

In [ ]:
# inspecting the parameters of the second fully-connected layer
net[2], net[2].state_dict()

(Linear(in_features=8, out_features=1, bias=True),
 OrderedDict([('weight',
               tensor([[-0.2315, -0.2047,  0.2872,  0.1532,  0.3241, -0.0740, -0.1817, -0.3398]])),
              ('bias', tensor([-0.1596]))]))

It contains two parameters, corresponding to the layer's weight and bias.

### Targeted Parameters

In [8]:
type(net[2].bias), net[2].bias.data # extracting the bias from the second layer

(torch.nn.parameter.Parameter, tensor([-0.1596]))

In [9]:
net[2].bias.grad == None # the gradient has not been calculated yet

True

### All parameters at once

In [10]:
[(name, param.shape) for name, param in net.named_parameters()] # inspecting all parameters of the network

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

## Tied Parameters

In [12]:
# running the forward pass
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))
net(X)
# check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# make sure that they are actually the same object rather than just having the same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


This example shows that the parameters of the second and third layer are tied. They are not just equal, they are represented by the same exact tensor. Thus, if we change one of the parameters, the other one changes, too.

